In [57]:
import os
import pandas as pd
import numpy as np
import re
import string
import demoji
import spacy
import pycld2 as cld2
from textblob import TextBlob



In [34]:
data_path = os.listdir('Datasets/Tweet Data')
data_path.sort()
data_path = ['Datasets/Tweet Data/{}'.format(file) for file in data_path]
data_path = pd.DataFrame(data_path,columns=['File Path'])

try:
    data_path = pd.concat([data_path,pd.DataFrame(data_path['File Path'].str.split(r'_',expand=True)[2])],axis=1)
    data_path = pd.concat([data_path,data_path[2].str.split(r'\.',expand=True)],axis=1,)
    data_path = data_path[['File Path',0]]
    data_path.columns = ['File Path', 'Counter']
    data_path['Counter'] = data_path['Counter'].apply(lambda x: int(x))
    
except:
    print('error')

In [11]:
from distutils.log import error
day = 0

for i,file in enumerate(data_path.values):
    if np.mod(file[1],1440)==0:
        try:
            day +=1
            print(day,i,file[1],pd.read_csv(file[0],engine='python',index_col=0).shape)
            
        except error as e:
            print(e)

1 22 1440 (19357, 11)
2 47 2880 (20904, 11)
3 71 4320 (20553, 11)
4 96 5760 (20568, 11)
5 121 7200 (20319, 11)
6 145 8640 (20308, 11)
7 169 10080 (20598, 11)
8 193 11520 (20454, 11)
9 217 12960 (20477, 11)
10 240 14400 (20265, 11)
11 264 15840 (20371, 11)
12 289 17280 (20204, 11)
13 313 18720 (20368, 11)
14 337 20160 (20563, 11)
15 361 21600 (20744, 11)
16 384 23040 (20156, 11)
17 408 24480 (20535, 11)
18 432 25920 (20602, 11)
19 456 27360 (20306, 11)
20 481 28800 (20125, 11)
21 505 30240 (20490, 11)
22 529 31680 (20497, 11)
23 552 33120 (20452, 11)
24 577 34560 (20588, 11)
25 601 36000 (20356, 11)
26 625 37440 (20595, 11)
27 649 38880 (20556, 11)
28 673 40320 (20648, 11)
29 697 41760 (20663, 11)
30 721 43200 (20452, 11)
31 743 44640 (20173, 11)
32 744 44640 (20187, 11)
33 766 46080 (20411, 11)
34 790 47520 (20572, 11)
35 813 48960 (20270, 11)
36 837 50400 (20156, 11)
37 861 51840 (20398, 11)
38 885 53280 (20335, 11)
39 908 54720 (19922, 11)
40 931 56160 (19966, 11)
41 956 57600 (20128

- Remove @[someone] - Done
- Remove string 'RT'- Done
- Remove new line - Done
- Remove punctuation (Remove # only; keep the string) - Done
- Remove links to http - Done
- Replace emojis by description - Done
- Remove text with numbers in them

In [35]:
def pre_process_tweet (text):
    #Remove all links starting with http...
    text = re.sub('https?:\/\/.*[\r\n]*','',text)
    #Remove RT
    text = re.sub('^RT[\s]+','',text)
    #Remove @[User]
    text = re.sub('@[^ ]+', '', text)
    #Remove Punctuation first
    text = text.translate(str.maketrans('','', string.punctuation))
    #Convert text to lower case
    text = text.lower()
    #Removing any text with numbers
    text = re.sub('[0-9]+.+ ','',text)
    #Remove new line
    text = re.sub('\n',' ',text)
    #Reducing whitespaces to one everywhere
    text = re.sub('\s+',' ',text)
    
    return text

In [36]:
unproc_data = pd.read_csv(data_path['File Path'][0],index_col=0)

In [37]:
print(unproc_data['text'][2])

Iae galera, estão acompanhando essa alta no mercado? 
#Bitcoin só está fazendo a gente criar mais expectativa em haha

#32K 🚀 #criptomoedas


In [38]:
pre_process_tweet(unproc_data['text'][2])

'iae galera estão acompanhando essa alta no mercado bitcoin só está fazendo a gente criar mais expectativa em haha criptomoedas'

In [39]:
text2 = demoji.replace_with_desc((pre_process_tweet(unproc_data['text'][2])),sep=' ')
print(text2)

iae galera estão acompanhando essa alta no mercado bitcoin só está fazendo a gente criar mais expectativa em haha criptomoedas
